In [34]:
import pandas as pd

In [35]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [36]:
# загружаем данные по ppp,  валюте и континенту

url_gdp = 'https://raw.githubusercontent.com/datasets/ppp/master/data/ppp-gdp.csv'
gdf_df = pd.read_csv(url_gdp,  on_bad_lines='skip')
gdf_df = gdf_df[gdf_df['Year'] == 2020]
gdf_df['Country'] = gdf_df['Country'].apply(str.upper)
gdf_df.replace({'CZECH REPUBLIC': 'CZECHIA'}, inplace=True)

currancy_codes_df = pd.read_csv('https://datahub.io/core/currency-codes/r/codes-all.csv')[['Entity','AlphabeticCode', 'Currency']]
currancy_codes_df.columns = ['Country', 'CurrancyCode', 'Currency']
currancy_codes_df = currancy_codes_df.drop_duplicates(['Country'])

continents_df = pd.read_csv('https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv')
continents_df['Country'] = continents_df['Country'].apply(str.upper)
continents_df.replace({'CZ': 'CZECHIA'}, inplace=True)


In [37]:
# загружаем датасет с kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

file_name = 'netflix%20price%20in%20different%20countries.csv'
api.dataset_download_file('prasertk/netflix-subscription-price-in-different-countries', file_name=file_name,  path='data/')

fees_df = pd.read_csv('./data/' + file_name ,  on_bad_lines='skip')
fees_df['Country'] = fees_df['Country'].apply(str.upper)
fees_df.replace({'RUSSIA': 'RUSSIAN FEDERATION'}, inplace=True)


In [38]:
df = (fees_df
      .merge(continents_df)
      .merge(currancy_codes_df)
      .merge(gdf_df[['Country', 'PPP']])
     )

In [39]:
df.head()

,Country,Total Library Size,No. of TV Shows,No. of Movies,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($),Continent,CurrancyCode,Currency,PPP
0,ARGENTINA,4760,3154,1606,3.74,6.30,9.26,South America,ARS,Argentine Peso,28.673797
1,AUSTRIA,5640,3779,1861,9.03,14.67,20.32,Europe,EUR,Euro,0.764401
2,BOLIVIA,4991,3155,1836,7.99,10.99,13.99,South America,BOP,Peso boliviano,2.595625
3,BULGARIA,6797,4819,1978,9.03,11.29,13.54,Europe,BGN,Bulgarian Lev,0.702639
4,CHILE,4994,3156,1838,7.07,9.91,12.74,South America,CLP,Chilean Peso,418.432299


Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от валюты страны. 


In [40]:
df.groupby(['CurrancyCode', 'Currency'])[['Cost Per Month - Basic ($)', 
                            'Cost Per Month - Standard ($)', 
                            'Cost Per Month - Premium ($)']].mean().sort_values('Cost Per Month - Premium ($)')

,,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($)
CurrancyCode,Currency,,,
TRY,Turkish Lira,1.970000,3.000000,4.02
INR,Indian Rupee,2.640000,6.610000,8.60
ARS,Argentine Peso,3.740000,6.300000,9.26
COP,Colombian Peso,4.310000,6.860000,9.93
BRL,Brazilian Real,4.610000,7.110000,9.96
PEN,Sol,6.110000,8.560000,11.01
UAH,Hryvnia,5.640000,8.460000,11.29
THB,Baht,8.340000,10.430000,12.52
ZAR,Rand,6.260000,10.050000,12.58


Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от континента


In [41]:
df.groupby(['Continent'])[['Cost Per Month - Basic ($)', 
                            'Cost Per Month - Standard ($)', 
                            'Cost Per Month - Premium ($)']].mean().sort_values('Cost Per Month - Premium ($)')

,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($)
Continent,,,
South America,6.566667,9.466667,12.395556
Africa,6.260000,10.050000,12.580000
Asia,7.392222,10.353333,13.086667
North America,7.900000,11.454000,14.648000
Oceania,8.320000,12.325000,16.665000
Europe,9.221538,13.168846,17.375769


Сколько стоит подписка относительно среднего дохода (базовая, стандартная, премиум) в Чехии?


In [42]:
df_czk = df[df['Country'] == 'CZECHIA']
df_czk[['Cost Per Month - Basic ($)', 
              'Cost Per Month - Standard ($)', 
              'Cost Per Month - Premium ($)']] / df_czk['PPP'].iloc[0]

,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($)
8,0.697577,0.907719,1.117862


Какие топ-5 стран имеют самую большую доступную коллекцию сериалов?

In [43]:
df.sort_values('Total Library Size', ascending=False).head(5)['Country'].values

array(['CZECHIA', 'HUNGARY', 'BULGARIA', 'IRELAND', 'LITHUANIA'],
      dtype=object)

Какой процент среднего дохода россиянина будет составлять подписка Netflix?

In [44]:
(df[df['Country'] == 'RUSSIAN FEDERATION']['Cost Per Month - Standard ($)']
 / df[df['Country'] == 'RUSSIAN FEDERATION']['PPP']).values[0] * 100

41.88435066196593

На каком месте будет Россия, если отсортировать страны по эффективности подписки (Cost Per Title)


In [45]:
df['cpt'] = df['Cost Per Month - Standard ($)'] / df['Total Library Size']
df = df.loc[df.cpt.argsort()[::-1]].reset_index(drop=True)
print('Russia rank:', df[df['Country'] == 'RUSSIAN FEDERATION'].index.values[0])

Russia rank: 36


В какой стране лучше всего брать VPN для того, чтобы платить за Netflix меньше всего?

In [46]:
print('Best for VPN:', df.sort_values('Cost Per Month - Standard ($)')['Country'].values[0])

Best for VPN: TURKEY
